# 0. Request google earth engine access

In [10]:
##https://earthengine.google.com/
##use your google account

# 1. setup docker

In [2]:
###### see here:  http://earthenginesummit2016.earthoutreach.org/training-materials 

#1.1 install docker
#https://www.docker.com/
#1.2 if you do not install virtual machine yet, install it from here: https://www.virtualbox.org/wiki/Downloads
#1.3 create virtual machine
# see here:  (EN) https://docs.docker.com/machine/get-started/  or (JP) http://docs.docker.jp/machine/get-started.html  or  
## from terminal
#docker-machine create --driver virtualbox default
## you now find default machine
#docker-machine ls

# 2. setup docker container

In [4]:
#2.1 (OSX) Start up the docker-machine VM
#docker-machine start
#eval $(docker-machine env default)
#2.2 Start up the Docker container
#docker run -d -P tylere/docker-debian-python3-scipy-notebook
#2.3 Determine the IP and port for constructing a URL for viewing the notebooks
#docker-machine ip default
#docker ps

# 3. access from ipython notebook

In [7]:
#see here: https://docs.google.com/presentation/d/1N7W09Njvz9ROrarp8SspA0kSC17Z0bzBbzTxCxskdYI/edit#slide=id.g14e75e974e_0_664
#3.1 run jupyternotebook
#from terminal
#jupyter notebook
#Select New -> Terminal
#Type in:
#earthengine authenticate

#git clone https://github.com/tylere/docker-tmpnb-ee.git 
#Follow the instructions to complete authentication.
#go to New -> Python[root]
#	>>> import ee
#	>>> ee.Initialize()

In [16]:
%matplotlib inline
import ee
ee.Initialize()

In [20]:
## try to map SRTM
srtm = ee.Image("CGIAR/SRTM90_V4")
url = srtm.getThumbUrl()
print(url)

https://earthengine.googleapis.com/api/thumb?thumbid=f80f7287fc7bd84d2b68869865d7fdcf&token=d383bb9e6f1575119252ffb8cca8965b


What is returned by the .getInfo() command is a Python dictionary. If needed, we could parse out this information and make use of it in our analysis.

In [21]:
from IPython.display import Image
Image(url=url)

In [15]:
## see list of available commands
image_object = ee.Image()
[method for method in dir(image_object) if not method.startswith('_')]

['And',
 'Not',
 'Or',
 'abs',
 'acos',
 'add',
 'addBands',
 'args',
 'arrayAccum',
 'arrayDimensions',
 'arrayDotProduct',
 'arrayFlatten',
 'arrayGet',
 'arrayLength',
 'arrayLengths',
 'arrayMask',
 'arrayProject',
 'arrayReduce',
 'arrayRepeat',
 'arraySlice',
 'arraySort',
 'arrayTranspose',
 'aside',
 'asin',
 'atan',
 'atan2',
 'bandNames',
 'bandTypes',
 'bilinearSample',
 'bitCount',
 'bitsToArrayImage',
 'bitwiseAnd',
 'bitwiseNot',
 'bitwiseOr',
 'bitwiseXor',
 'bitwise_and',
 'bitwise_not',
 'bitwise_or',
 'bitwise_xor',
 'byte',
 'cast',
 'cat',
 'cbrt',
 'ceil',
 'changeProj',
 'clamp',
 'classify',
 'clip',
 'clipToCollection',
 'cluster',
 'combine_',
 'connectedComponents',
 'connectedPixelCount',
 'constant',
 'convolve',
 'copyProperties',
 'cos',
 'cosh',
 'cumulativeCost',
 'date',
 'derivative',
 'digamma',
 'displace',
 'displacement',
 'distance',
 'divide',
 'double',
 'encode',
 'entropy',
 'eq',
 'erf',
 'erfInv',
 'erfc',
 'erfcInv',
 'exp',
 'expression',


In [23]:
## following commands are same in EE 101.ipynb in docker-tmpnb-ee
#Earth Engine 101
###again
srtm = ee.Image("CGIAR/SRTM90_V4")

By default, the .getThumbUrl() method returns the entire extent of the image, which in this case is global. We can also specify a region, to show a smaller area.

In [24]:
point = ee.Geometry.Point(-122.0918, 37.422)
region_bay_area = point.buffer(50000).bounds().getInfo()['coordinates']
Image(url=srtm.getThumbUrl({'min':0, 'max':1000, 'region':region_bay_area}))

# Load and Filter an Image Collection
So far we have been working with a single image, but there are also interesting datasets that are distributed as a series of images (such as images collected by satellite). Head back to the Earth Engine Public Data Catalog, search for landsat 8 toa, and load up the data description page for the USGS Landsat 8 TOA Reflectance (Orthorectified) dataset. The ID for this Image Collection is LANDSAT/LC8_L1T_TOA.

In [26]:
# Create a reference to the image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
# Filter the collection down to a two week period
filtered = l8.filterDate('2013-05-01', '2013-05-15');
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.mosaic()
# Define a region roughly covering California
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(500000).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=l8_image.getThumbUrl({'region':region_california}))

# Playing with Image Bands
Using the default image visualization parameters, that doesn't look like much. So we add some visualization data, to display a true color image.

In [27]:
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

And by changing the bands displayed, we can also display a false color image.

In [28]:
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B5,B4,B3',
    'min':0,
    'max':0.3
}))

# Play with Reducing Image Collections
Next expand the date range to cover an entire year, so that there are many overlapping images. We will continue to use the .mosaic() reducer, which retains the last (most recent) pixels in areas of image overlap. Clouds are readily apparent.

In [29]:
filtered = l8.filterDate('2013-01-01', '2014-01-01')

### ImageCollection.mosaic Reducer¶

In [30]:
l8_image = filtered.mosaic()
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

### ImageCollection.median Reducer

In [31]:
l8_image = filtered.median()
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

### ImageCollection.min Reducer

In [32]:
l8_image = filtered.min()
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

### ImageCollection.max Reducer

In [33]:
l8_image = filtered.max()
Image(url=l8_image.getThumbUrl({
    'region':region_california,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))